In [ ]:
from TimeSub.estimation import cr_data, create_net_class
from TimeSub.prediction import prediction_ability
import numpy as np
import pandas as pd

In [8]:
time_breaks = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25]

In [ ]:
cov = np.load("data/covariates_TI_train.npy")
data = pd.read_csv("data/sample_TI_train.csv")
# cov = np.load("data/covariates_TV_train.npy")
# data = pd.read_csv("data/sample_TV_train.csv")
t_vec = data['T']
T = data['T'].to_numpy().reshape(-1, 1)
interval_indices = np.searchsorted(time_breaks, t_vec, side='right')-1
Z = cov
n = Z.shape[0]
d = Z.shape[1]
TX = np.zeros((n, n, d+1))
TX[:,:,0] = T
for i in range(d):
    TX[:, :, i+1] = cov[:, i].reshape(1, n)
# Z = cov[:,interval_indices,:].transpose(1, 0, 2)
# TX = np.zeros((Z.shape[0], Z.shape[1], Z.shape[2]+1))
# TX[:,:,0] = T
# TX[:, :, 1:] = Z
Delta = (data['event_type']>0).to_numpy(dtype=float).reshape(-1, 1)
epsilon = (data['event_type']).to_numpy(dtype=float).reshape(-1, 1)
data_train = cr_data(T,Delta,epsilon,Z,TX,cov)
data_train.to_torch()



In [ ]:
cov = np.load("data/covariates_TI_test.npy")
data = pd.read_csv("data/sample_TI_test.csv")
# cov = np.load("data/covariates_TV_test.npy")
# data = pd.read_csv("data/sample_TV_test.csv")
t_vec = data['T']
T = data['T'].to_numpy().reshape(-1, 1)
interval_indices = np.searchsorted(time_breaks, t_vec, side='right')-1
Z = cov
n = Z.shape[0]
d = Z.shape[1]
TX = np.zeros((n, n, d+1))
TX[:,:,0] = T
for i in range(d):
    TX[:, :, i+1] = cov[:, i].reshape(1, n)
# Z = cov[:,interval_indices,:].transpose(1, 0, 2)
# TX = np.zeros((Z.shape[0], Z.shape[1], Z.shape[2]+1))
# TX[:,:,0] = T
# TX[:, :, 1:] = Z
Delta = (data['event_type']>0).to_numpy(dtype=float).reshape(-1, 1)
epsilon = (data['event_type']).to_numpy(dtype=float).reshape(-1, 1)
data_test = cr_data(T,Delta,epsilon,Z,TX,cov)
data_test.to_torch()

In [ ]:
data_train.structure_test(B_seeds = [100000+(i+1)*10 for i in range(100)],
                          learning_rate = 1e-3, 
                          num_epoch = 300, 
                          num_epoch_ = 50, 
                          num_epoch_B = 50, 
                          learning_rate_B = 5e-2, 
                          Nets = [create_net_class(6, 12, 3)],
                          max_batch_size = 2000 ,
                          valid_rate = 0.5,
                          Nets_TI = [create_net_class(5, 12, 3)],
                          log = None,
                          num_model = 5, 
                          num_model_B = 5):
# Perform structure test.
# B_seeds: list of bootstrap seeds.
# learning_rate: learning rate for initial model training.
# num_epoch: number of epochs for initial model training.
# num_epoch_: number of epochs for model training during test statistic calculation.
# num_epoch_B: number of epochs for model training during bootstrap test statistic calculation.
# learning_rate_B: learning rate for model training during test statistic calculation.
# Nets: list of neural network classes for the main model.
# max_batch_size: maximum batch size for training.
# valid_rate: proportion of data used for validation.
# Nets_TI: list of neural network classes for the null model.
# log: file path for logging results.
# num_model: number of random initializations for model training during test statistic calculation.
# num_model_B: number of random initializations for model training during bootstrap test statistic calculation
plr,time = data_train.plr_st
plr_B_time_list = data_train.plr_st_B
# plt: statistic value.
# time: time taken for the structure test.
# plt_B_time_list: list of statistic values from bootstrap samples.


In [ ]:
data_train.linear(num_epoch = 200, 
                  learning_rate = 0.01)
# the original subdistribution hazard model (SHM) 
# num_epoch: number of epochs for training.
# learning_rate: learning rate for optimization.
data_train.spline(num_spline = 10,
               num_epoch = 500,
               learning_rate = 0.01)
# the spline-based time-varying coeffcient subdistribution hazard model (TSHM) using cubic splines.
# num_spline: number of spline basis functions.
# num_epoch: number of epochs for training.
# learning_rate: learning rate for optimization.
data_train.nn_ti(Nets = [create_net_class(5, 10, 1),create_net_class(5, 20, 3)],
            batch_num = 1,
            learning_rate1=[1e-2,1e-3],
            random_num = 5,
            num_epoch = 300)
# the nonparametric subdistribution hazard model (NSHM) using neural networks.
# Nets: list of neural network classes to be used. 
# batch_num: number of batches for training.
# learning_rate1: list of learning rates for different stages of training. Corresponds to Nets.
# random_num: number of random initializations for model selection.
# num_epoch: number of epochs for training.
data_train.nn_tv(Nets = [create_net_class(6, 10, 1),create_net_class(6, 20, 3)],
              batch_num = 1,
              learning_rate1=[1e-2,1e-3],
              random_num = 5,
             num_epoch = 300)
# time-varying nonparametric subdistribution hazard model (TNSHM) using neural networks.
# Nets: list of neural network classes to be used. (6,10,1) means input size 6, hidden size 10, depth 1. 
# batch_num: number of batches for training.
# learning_rate1: list of learning rates for different stages of training. Corresponds to Nets.
# random_num: number of random initializations for model selection.
# num_epoch: number of epochs for training.
g_torch_linear =  data_train.models['linear'][0](data_test)
g_torch_spline =  data_train.models['spline'][0](data_test)
g_torch_nntv =  data_train.models['nn_tv'][0](data_test)
g_torch_nnti =  data_train.models['nn_ti'][0](data_test)
# g_torch_linear: predicted log subdistribution hazard from SHM.
# g_torch_spline: predicted log subdistribution hazard from TSHM.
# g_torch_nntv: predicted log subdistribution hazard from TNSHM.
# g_torch_nnti: predicted log subdistribution hazard from NSHM.
gc_linear,auc_linear =  prediction_ability(data_test,g_torch_linear,g0_torch=g_torch_nntv) 
gc_spline,auc_spline =  prediction_ability(data_test,g_torch_spline,g0_torch=g_torch_nntv)
gc_nntv,auc_nntv =  prediction_ability(data_test,g_torch_nntv,g0_torch=g_torch_nntv)
gc_nnti,auc_nnti =  prediction_ability(data_test,g_torch_nnti,g0_torch=g_torch_nntv)
# gc_linear: GC for SHM.
# auc_linear: AUC for SHM.
# gc_spline: GC for TSHM.
# auc_spline: AUC for TSHM.
# gc_nntv: GC for TNSHM.
# auc_nntv: AUC for TNSHM.
# gc_nnti: GC for NSHM.
# auc_nnti: AUC for NSHM.

In [ ]:
data_train.significance_test(cov_index = 5,
                             B_seeds,learning_rate = 1e-3, 
                             num_epoch = 300, 
                             num_epoch_ = num_B, 
                             num_epoch_B = num_B, 
                             learning_rate_B = learning_B, 
                             Nets = [create_net_class(6, 12, 3)],
                             max_batch_size = 2000,
                             valid_rate = 0.5,
                             Nets_null = [create_net_class(5, 12, 3)],
                             log = None,
                             num_model = 5, 
                             num_model_B = 5):
# Perform significance test for a specific covariate.
# cov_index: index of the covariate to be tested.
# B_seeds: list of bootstrap seeds.
# learning_rate: learning rate for initial model training.
# num_epoch: number of epochs for initial model training.
# num_epoch_: number of epochs for model training during test statistic calculation.
# num_epoch_B: number of epochs for model training during bootstrap test statistic calculation.
# learning_rate_B: learning rate for model training during test statistic calculation.
# Nets: list of neural network classes for the main model.
# max_batch_size: maximum batch size for training.
# valid_rate: proportion of data used for validation.
# Nets_null: list of neural network classes for the null model.
# log: file path for logging results.
# num_model: number of random initializations for model training during test statistic calculation.
# num_model_B: number of random initializations for model training during bootstrap test statistic calculation
plr,time = data_train.plr_si
plr_B_time_list = data_train.plr_si_B